In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,  img_to_array
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [2]:
# gpu 연결 체크
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
# True

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# 윈도우 케라스에서 cpu 멀티프로세싱하기
# 멀티프로세싱은 윈도우에서 생성기를 지원하지 않음(리눅스 가능)
# 그래서 시퀀스로 병렬 생성기 작동

In [4]:
# 이미지 증강기 인스턴스 생성
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 360, # 임의의 회전 각도 범위
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True,
                                   vertical_flip = True,
                                   fill_mode = 'nearest') # 입력 경계 밖의 픽셀을 무엇으로 채울지

In [5]:
# 디렉토리에 대한 경로를 사용하고 증강 데이터 배치를 생성
# bachsize만큼 랜덤으로 추출
train_generator = train_datagen.flow_from_directory(
                  'resample', # 디렉토리
                  target_size = (150, 150), # 정수튜플, 이미지 크기 조정 h,w 디폴트 256, 256
                  #target_size = (420, 360),
                  batch_size = 16,  # 데이터 배치크기, default 32 
                  class_mode = 'sparse') # sparse는 1d 정수 레이블
train_generator
# DirectoryIterator생성 튜플은 모양 이 있는 이미지 배치를 포함하는 numpy 배열이고 해당 레이블의 numpy 배열입니다.

Found 23058 images belonging to 4 classes.


In [6]:
for data_batch, labels_batch in train_generator:
    # 데이터 하나만 체크해보자
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break


배치 데이터 크기: (16, 150, 150, 3)
배치 레이블 크기: (16,)


In [7]:
inputs = keras.Input(shape=(150, 150, 3), name="img")
#inputs = keras.Input(shape=(420, 360, 3), name="img")
x = layers.Conv2D(16, 3, 2, padding = 'same', activation="relu")(inputs)
x = layers.Conv2D(32, 3, 1, padding ='same', activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(64, 3, 1, activation="relu", padding="same")(x)
x = layers.Conv2D(64, 3, 2, activation="relu", padding="same")(x)
x = layers.Flatten()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(4)(x)
outputs = layers.Softmax()(x)

model = keras.Model(inputs, outputs, name="converea")
model.summary()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

Model: "converea"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 75, 75, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        36928     
                                                          

In [8]:
model.fit(train_generator, batch_size=16, epochs=1)
tf.saved_model.save(model, './model/converea')

1442/1442 [==============================] - 63s 41ms/step - loss: 0.4012 - accuracy: 0.8430


INFO:tensorflow:Assets written to: ./model/converea\assets


INFO:tensorflow:Assets written to: ./model/converea\assets


In [9]:
model.save_weights('./model/checkpoint')

In [10]:
loaded = tf.saved_model.load('./model/converea')

loaded(data_batch)

<tf.Tensor: shape=(16, 4), dtype=float32, numpy=
array([[9.4563468e-10, 5.4678764e-02, 9.4392705e-01, 1.3941592e-03],
       [7.5542240e-04, 8.6155653e-01, 1.3489306e-01, 2.7950178e-03],
       [9.3327235e-25, 5.3036724e-12, 3.6653112e-09, 1.0000000e+00],
       [3.9858091e-06, 9.9566597e-01, 4.3300167e-03, 1.1515383e-09],
       [7.6867419e-17, 5.1018153e-04, 9.9948967e-01, 1.4369765e-07],
       [9.3374687e-01, 6.5034725e-02, 7.1542989e-04, 5.0300546e-04],
       [4.1906734e-08, 3.4225531e-02, 9.5804036e-01, 7.7341492e-03],
       [1.1285094e-02, 9.8820299e-01, 5.1159383e-04, 4.3546885e-07],
       [1.5589712e-06, 9.9989927e-01, 9.9122328e-05, 9.7095400e-13],
       [1.7349999e-10, 1.1352651e-02, 9.8792404e-01, 7.2336290e-04],
       [2.4555298e-03, 9.9736470e-01, 1.7974577e-04, 3.0842401e-08],
       [1.1205381e-21, 8.5583294e-14, 4.2970008e-03, 9.9570304e-01],
       [6.8324320e-05, 9.9980921e-01, 1.2251809e-04, 9.2505265e-10],
       [9.7893952e-08, 9.3593293e-01, 6.4066820e-02, 1

In [ ]:
model.fit_generator(train_generator, batch_size=16, epochs=2, use_multiprocessing=True,
                         workers=6)
tf.saved_model.save(model, './model/converea')

In [3]:
import os
import json
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})

In [5]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead


In [ ]:
NUM_WORKERS = 4
# `tf.data.Dataset.batch`에는 전역 배치 크기를 지정해야 하기 때문에
# 여기서 배치 크기는 워커의 수를 곱한 크기로 늘려야 합니다. 
# 전에는 64였지만, 이제 128이 됩니다.
GLOBAL_BATCH_SIZE = 64 * NUM_WORKERS
train_datasets = train_generator.batch(GLOBAL_BATCH_SIZE)
with strategy.scope():
    multi_worker_model = build_and_compile_cnn_model()
multi_worker_model.fit(x=train_datasets, epochs=2)

# model.fit(train_generator, batch_size=16, epochs=2)
tf.saved_model.save(multi_worker_model, './model/converea1')
